<a href="https://colab.research.google.com/github/mrarringtonfh/FH_NewUserScripts/blob/main/FHStudentCleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [76]:
import csv
import random
import string
import datetime

def get_random_password():
  # Generates a random password
  words = [line.strip() for line in open("/content/drive/Shareddrives/Technology/ColabData/membean6.txt", "r")]
  word1 = random.choice(words).capitalize()
  word2 = random.choice(words).capitalize()[0:2]
  number = random.randint(100, 999)
  special = random.choice(string.punctuation)
  return f"{word1}{special}{number}"

def process_student_data(student):
  # Processes student data and returns a dictionary
  last_name = student["LastName"].replace(" ", "")
  short_last_name = last_name if len(last_name) < 5 else last_name[:5]

  campus = "EastCampusStudents" if int(student["GradYear"]) > 2030 else "WestCampusStudents"

  building = "LSStudents" if int(student["GradYear"]) > 2030 else (
      "MSStudents" if int(student["GradYear"]) > 2028 else "USStudents"
  )

  password = get_random_password()

  return {
      "givenName": student["PreferredName"],
      "initials": student["MiddleName"][0] if student["MiddleName"] else "",
      "sn": student["LastName"],
      "password": password,
      "description": f"Class of {student['GradYear']}",
      "department": f"Class of {student['GradYear']}",
      "OUName": f"OU={student['GradYear']},OU=Students,OU=FHS-Users,DC=FLINTHILL,DC=ORG",
      "userAccountControl": "512",
      "memberOf": f'"CN=Student,OU=Groups,OU=FHS-Users,DC=FLINTHILL,DC=ORG;CN={building},OU=Groups,OU=FHS-Users,DC=FLINTHILL,DC=ORG;CN={campus},OU=Groups,OU=FHS-Users,DC=FLINTHILL,DC=ORG"',
      "gappsOrgUnitPath": f"/Students/{student['GradYear']}",
      "gappsGroupEmail": f"{student['GradYear']}@flinthill.org",
      "templateName": "FHSStudentUsers",
      "userid" : student["UserID"],
      "username": student['FirstName'][0].lower() + last_name.lower(),
      "email": f"{student['FirstName'][0].lower()}{last_name.lower()}@flinthill.org",
      "ipphone": student['FirstName'][0].lower() + short_last_name.lower(),
  }

# Read student data from CSV
with open("/content/drive/Shareddrives/Technology/ColabData/NewStudents_FH.csv") as csvfile:
  reader = csv.DictReader(csvfile)
  student_data = [process_student_data(student) for student in reader]

# Get today's date in YYYYMMDD format
today = datetime.date.today().strftime("%Y%m%d")

# Construct the output filename with date prefix
output_filename = f"FHStudentsReadyForImport_{today}.csv"

with open("/content/drive/Shareddrives/Technology/ColabData/" + output_filename, "w", newline="") as csvfile:
    fieldnames = student_data[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(student_data)

print(f"Student data processed and written to {output_filename}")

Student data processed and written to FHStudentsReadyForImport_20241118.csv
